In [ ]:
import random
import math
import time
import sympy
from collections import deque
from tqdm import tqdm

In [ ]:
def func(a, b, c, u, v, p, q):
    if c < p // 2:
        c = pow(a * c, 1, p)
        u = pow(u + 1, 1, q)
    else:
        c = pow(b * c, 1, p)
        v = pow(v + 1, 1, q)
    return c, u, v

In [ ]:
def pollards_rho_algorithm(a, b, p, q):
    u = random.randint(0, p)
    v = random.randint(0, p)
    c = (pow(a, u, p) * pow(b, v, p)) % p
    d, n, m = c, u, v
    total_iterations = 0
    stack_size = 5
    begin_print_stack = deque(maxlen = stack_size)
    end_print_stack = deque(maxlen = stack_size)
    print(f"Step: 0, c: {c}, loga_c: {u}+{v}x, d: {d}, loga_d: {n}+{m}x")
    for i in tqdm(range(10000000000), desc="Iterations", unit=" iterations"):
        c, u, v = func(a, b, c, u, v, p, q)
        d, n, m = func(a, b, d, n, m, p, q)
        d, n, m = func(a, b, d, n, m, p, q)
        if i < 6:
            begin_print_stack.append(f"Step: {i}, c: {c}, loga_c: {u}+{v}x, d: {d}, loga_d: {n}+{m}x")
        else:
            end_print_stack.append(f"Step: {i}, c: {c}, loga_c: {u}+{v}x, d: {d}, loga_d: {n}+{m}x")
        if c == d:
            total_iterations = i
            break
    if m > v:
        v = m - v
        u = u - n
    else:
        v = v - m
        u = n - u
    if u < 0:
        u = q + u
    while u % v != 0:
        u = u + q
    for line in begin_print_stack:
        print(line)
    for line in end_print_stack:
        print(line)
    print(f"Total iterations = {total_iterations}")
    print(f"Result: {u // v}")

In [ ]:
def gelfond_algorithm(a, b, p, q):
    m = int(math.sqrt(q) + 1)
    print(f"s = {m}")
    base = dict()
    stack_size = 5
    begin_print_stack = deque(maxlen = stack_size)
    end_print_stack = deque(maxlen = stack_size)
    for j in tqdm(range(m), desc="Base", unit=" elements"):
            x_j = (b * pow(a, j, p)) % p
            base[x_j] = j
            if j < 5:
                begin_print_stack.append((j, x_j))
            else:
                end_print_stack.append((j, x_j))
    gamma = pow(a, m, p)
    for i in tqdm(range(1, m + 1), desc="Sequence", unit=" elements"):
        if gamma in base:
            print("Base:")
            for j, x_j in begin_print_stack:
                print(f"k: {j}, value[k]: {x_j}")
            for j, x_j in end_print_stack:
                print(f"k: {j}, value[k]: {x_j}")
            print(f"t = {i}")
            print(f"Result: {(i * m - base[gamma]) % p}")
            return
        gamma = (gamma * pow(a, m, p)) % p
    print("Result not found.")

In [ ]:
def create_base(n):
    lk = 0
    base = [-1, 2]
    for i in range(3, n + 1, 2):
        if sympy.isprime(i):
            base.append(i)
            lk = lk + 1
    # amount of elements in base
    return base

# Gaussian elimination
def gauss(A, B, X, n):
    # A - alfa array
    # B - u - array
    # X - x - array
    # n = h + 1

    k = 0
    while k < n:
        max_elem = abs(A[k][k])
        index = k
        for i in range(k + 1, n):
            if abs(A[i][k]) > max_elem:
                max_elem = abs(A[i][k])
                index = i
        if max_elem == 0.0:
            return -1
        for j in range(n):
            temp = A[k][j]
            A[k][j] = A[index][j]
            A[index][j] = temp
        temp = B[k]
        B[k] = B[index]
        B[index] = temp
        for i in range(k, n):
            temp = A[i][k] 
            if abs(temp) == 0:
                continue
            for j in range(n):
                A[i][j] /= temp
            B[i] /= temp
            if i == k:
                continue
            for j in range(n):
                A[i][j] -= A[k][j]
            B[i] -= B[k]
        k += 1
    for k in range(n - 1, -1, -1):
        X[k] = B[k]
        for i in range(k):
            B[i] -= A[i][k] * X[k]
    return 1

def is_number_B_smooth(num, vector, size, base):
    for k in range(size):
        vector[k] = 0
    i = 1
    while i != size:  
        if num % base[i] == 0:
            num /= base[i]
            vector[i] += 1
            if num == 1:
                return 1
        else:
            i += 1
    return 0

def create_matrix(h):
    u_vector = []
    x_vector = []
    beta_vector = []  # beta array with degrees
    alfa_matrix = []  # alfa array with degrees
    for k in range(h + 1):
        alfa_matrix.append([0] * (h + 1))
        u_vector.append(0)
        beta_vector.append(0)
        x_vector.append(0)
    return u_vector, x_vector, beta_vector, alfa_matrix

In [ ]:
def base_decomposition_method(a, b, p, q):
    # Step 1
    ln_p = math.log(p, math.e)
    limiter = math.sqrt(int(pow(math.e, math.sqrt(ln_p * math.log(ln_p, math.e)))))

    base = create_base(int(limiter))
    h = len(base) - 1

    print (f"ph: {int(limiter)}")
    print (f"Size of base B: {h + 1}")
    print (f"Created base B:{base}")
    while True:
        # Step 2
        u_vector, x_vector, beta_vector, alfa_matrix = create_matrix(h) # Initialization of matrix and vectors
        i = 0
        
        for i in tqdm(range(h + 1)):
            flag_smooth = 0
            ui = random.randint(2, p - 1)
            bi = pow(a, ui, p)

            # If bi is not B-smooth then check if p - bi is B-smooth
            if is_number_B_smooth(bi, alfa_matrix[i], h + 1, base) == 0:
                if is_number_B_smooth(p - bi, alfa_matrix[i], h + 1, base) == 1:
                    flag_smooth = 1
            else: flag_smooth = 1
            if flag_smooth:
                if ui in u_vector:
                    for j in range(h + 1):
                        alfa_matrix[i][j] = 0
                else:
                    alfa_matrix[i][0] = 1
                    u_vector[i] = ui
                    '''print(f"ui = {ui}")
                    print(f"Alfa matrix[i]: {alfa_matrix[i]} ")
                    print(f"bi = {bi}")'''
                    i += 1
        # Step 3
        v = 1
        while True:
            bv = pow(b, v, p)
            if is_number_B_smooth(bv, beta_vector, h + 1, base) == 1:
                break
            else:
                v += 1
        print (f"v = {v}")

        # Step 4-5
        # Gaussian elimination
        status = gauss(alfa_matrix, u_vector, x_vector, h + 1)
        if status < 0:
            print("No solution found")
            continue

        xv = 0
        for i in range(h + 1):
            xv = (xv + beta_vector[i] * x_vector[i]) % q

        if int(xv) != xv:
            print ("Incorrect solution")
            continue
            #return -1
        print(f"xv = {xv}")

        while xv % v != 0:
            xv += q

        x = (xv / v) % q

        if pow(a, int(x), p) != b:
            print("Incorrect solution")
            continue

        print (f"x:{x}")
        return 1

In [ ]:
print(f"Pollards Rho Algorithm")
Time = time.time()
pollards_rho_algorithm(13, 3, 50091786122438801387, 25045893061219400693)
Time = time.time() - Time
print(f"Elapsed time: {Time}")

In [ ]:
print(f"Gelfond algorithm")
Time = time.time()
gelfond_algorithm(13, 3, 50091786122438801387, 25045893061219400693)
Time = time.time() - Time
print(f"Elapsed time: {Time}")

In [ ]:
print(f"Base of decomposition algorithm")
Time = time.time()
base_decomposition_method(13, 3, 50091786122438801387, 25045893061219400693)
Time = time.time() - Time
print(f"Elapsed time: {Time}")